<a href="https://colab.research.google.com/github/DurhamARC/openpose-music/blob/master/OpenPose_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pose Detection with OpenPose

This notebook uses an open source project [CMU-Perceptual-Computing-Lab/openpose](https://github.com/CMU-Perceptual-Computing-Lab/openpose.git) to detect/track multi person poses on a given youtube video.

For other deep-learning Colab notebooks, visit [tugstugi/dl-colab-notebooks](https://github.com/tugstugi/dl-colab-notebooks).


##Install OpenPose

In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
!rm -rf openpose
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.17/cmake-3.17.2-Linux-x86_64.tar.gz
  !tar xfz cmake-3.17.2-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  # --recursive necessary in the line below, as otherwise you can (sometimes) get "lpthreads" errors in cmake ("undefined reference to `pthread_create'" etc). See, for example, https://github.com/facebookarchive/caffe2/issues/1234
  !sed -i 's/execute_process(COMMAND git checkout --recursive master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # build openpose
  # CUDA
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON .. && make -j`nproc`
  # CPU 
  #!cd openpose && rm -rf build || true && mkdir build && cd build && cmake -DBUILD_PYTHON=ON -DGPU_MODE=CPU_ONLY -DUSE_MKL=OFF .. && make -j`nproc`



##Tests

###Run Python API test examples

In [0]:
!pwd
print("Running Python API example.")
print("The imshow command in that example will lead to a 'cannot connect to X server' error in the end, which can be supressed by changing it to imwrite, for example.")
!cd openpose/build/examples/tutorial_api_python && python3 01_body_from_image.py
print("Another example.")
!cd openpose/build/examples/tutorial_api_python && python3 06_face_from_image.py
print("And another.")
!cd openpose/build/examples/tutorial_api_python && python3 09_keypoints_from_heatmaps.py

### Detect poses on a test video

Download video from Dropbox

In [0]:
!rm -rf full_video.mp4
videolink="https://www.dropbox.com/sh/fcbe7ebrvuutfgh/AABP8D9S1CPbLH__AOgqSkKTa/Apoorva_experiment_clips/Apoorva_raga_3a_muted.mp4?dl=0"
#videolink="https://www.dropbox.com/sh/fcbe7ebrvuutfgh/AACEKKDfkyN4vEgLJWZiGqYra/Raga%20edited%20clips/1a.mp4?dl=0"
!wget -O full_video.mp4 $videolink 

Access the video, cut the first 5 seconds and do the pose detection on that 5 seconds:

In [0]:
!rm -rf video.mp4
# cut the first 5 seconds
!ffmpeg -y -loglevel info -i "full_video.mp4" -t 5 video.mp4
# detect poses on the these 5 seconds
!rm -f openpose.avi
!cd openpose && ./build/examples/openpose/openpose.bin --video ../video.mp4 --write_json ./output/ --display 0 --part-candidates --write_video ../openpose.avi
# convert the result into MP4
!ffmpeg -y -loglevel info -i openpose.avi output.mp4

Finally, visualize the result:

In [0]:
import numpy as np
import os
import cv2
import json
print('OpenCV - version: ',cv2.__version__)
import pandas as pd

# video file
cap = cv2.VideoCapture('video.mp4')

def get_vid_properties(): 
    width = int(cap.get(3))  # float
    height = int(cap.get(4)) # float
    cap.release()
    return width,height
  
print('Video Dimensions: ',get_vid_properties())

# Load keypoint data from JSON output
column_names = ['x', 'y', 'acc']

# Paths - should be the folder where Open Pose JSON output was stored
path_to_json = "openpose/output/"

# Import Json files, pos_json = position JSON
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print('Found: ',len(json_files),'json keypoint frame files')
count = 0

width,height = get_vid_properties()

# instanciate dataframes 
body_keypoints_df = pd.DataFrame()
left_shoulder_df = pd.DataFrame()
left_elbow_df = pd.DataFrame()

print('json files: ',json_files[0])   

# Loop through all json files in output directory
# Each file is a frame in the video
# If multiple people are detected - choose the most centered high confidence points
for file in json_files:

    temp_df = json.load(open(path_to_json+file))
    temp = []
    for k,v in temp_df['part_candidates'][0].items():
        
        # Single point detected
        if len(v) < 4:
            temp.append(v)
            #print('Extracted highest confidence points: ',v)
            
        # Multiple points detected
        elif len(v) > 4: 
            near_middle = width
            np_v = np.array(v)
            
            # Reshape to x,y,confidence
            np_v_reshape = np_v.reshape(int(len(np_v)/3),3)
            np_v_temp = []
            # compare x values
            for pt in np_v_reshape:
                if(np.absolute(pt[0]-width/2)<near_middle):
                    near_middle = np.absolute(pt[0]-width/2)
                    np_v_temp = list(pt)
         
            temp.append(np_v_temp)
            #print('Extracted highest confidence points: ',np_v_temp)
        else:
            # No detection - record zeros
            temp.append([0,0,0])
            
    temp_df = pd.DataFrame(temp)
    temp_df = temp_df.fillna(0)
    #print(temp_df)

    try:
        prev_temp_df = temp_df
        body_keypoints_df= body_keypoints_df.append(temp_df)
        left_shoulder_df = left_shoulder_df.append(temp_df.iloc[5].astype(int))
        left_elbow_df = left_elbow_df.append(temp_df.iloc[6].astype(int))

    except:
        print('bad point set at: ', file)
        
body_keypoints_df.columns = column_names
left_shoulder_df.columns = column_names
left_elbow_df.columns = column_names

body_keypoints_df.reset_index()
left_shoulder_df = left_shoulder_df.reset_index(drop = True)
left_elbow_df = left_elbow_df.reset_index(drop = True)

print('length of merged keypoint set: ',body_keypoints_df.size)

print(left_shoulder_df.head())
print(left_elbow_df.head())

In [0]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii'))) 

show_local_mp4_video('output.mp4', width=960, height=720)